In [ ]:
#tokenization

In [3]:
from transformers import AutoModel, AutoTokenizer
import torch
from torch.utils.data import Dataset

/home/shiva/GIT_C/NLP_multihead_multiclass/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "../models/biobert"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path)

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1244.49it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: ../models/biobert
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.decoder.bias               | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.decoder.weight             | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [1]:
sample_data = {
  "text": "short merge text ...",
  "modality": "MR",
  "vendor": "GE",
  "series_type": "FLAIR",
  "plane": "AX",
  "acquisition": "2D",
  "body": "brain",
  "contrast": "with"
}

In [2]:
encoded_sample = {
  "text": "short merge text ...",
  "label_modality": 0,
  "label_vendor": 1,
  "label_series_type": 2,
  "label_plane": 0,
  "label_acquisition": 1,
  "label_body": 3,
  "label_contrast": 0,
}

In [ ]:
'''
input: raw record : dict 
output: tokenized 

'''
class 


In [ ]:
#model construction

In [ ]:
#evaluation